# Tournaments playground

In [192]:
import math
import numpy as np

import typing
from collections.abc import Sequence, Callable

from qiskit import Aer
from qiskit import QuantumCircuit

Initially, I just wondered about common usage of single elimination tournament in games. The reason is that it just puts you in a group of top-2^n rather than creates a properly enumerated leaderboard. Though, both tournaments are not fair, I find the latter more satisfying as well as the algorithm behind it.

P.S. Tournaments here are only for 2^n players, without scores and draws, only wins and losses.

P.P.S. I'm not good with python type hints, they can be not absolutely right, but I don't think they are misleading

In [193]:
def count_check(participants):
    participants_count = len(participants)
    log2_count = math.log2(participants_count)

    if log2_count % 1 != 0:
        raise Exception("Participants count must be equal to the power of two")
    
    return participants_count, log2_count

In [194]:
def binary_tournament(participants: Sequence[object], comparator: Callable[[object, object], bool], verbose: bool = False) -> Sequence[object]:
    _, log2_count = count_check(participants)
    
    scores_binary = dict(zip(participants, ["" for _ in participants]))
    rounds_count = int(log2_count) 

    for round in range(rounds_count):
        if verbose:
            print(f"Round {round + 1}")
            print(f"Start scores: {scores_binary}")
        
        pairs = []
        pairs_checklist = set()

        for participant in participants:
            if participant in pairs_checklist:
                continue

            pairs_checklist.add(participant)
            score = scores_binary[participant]
            
            pair = next(p[0] for p in scores_binary.items() if score == p[1] and p[0] not in pairs_checklist)
            pairs_checklist.add(pair)
            pairs.append((participant, pair))
        
        if verbose:
            print(f"Assigned pairs: {pairs}")

        for pair in pairs:
            first_won = comparator(pair[0], pair[1])

            scores_binary[pair[0]] += str(int(first_won))
            scores_binary[pair[1]] += str(int(not first_won))

    scores_decimal = dict((p[0], int(p[1], 2)) for p in scores_binary.items())
    leaderboard = sorted(participants, key=lambda p: scores_decimal[p], reverse=True)

    if verbose:
        print()
        print(f"Final scores: {scores_binary}")
        print(f"Tournament results: {leaderboard}")

    return leaderboard

In [195]:
comparator = lambda x, y: x > y

participants = range(8)
binary_tournament(list(participants), comparator, verbose=True)

Round 1
Start scores: {0: '', 1: '', 2: '', 3: '', 4: '', 5: '', 6: '', 7: ''}
Assigned pairs: [(0, 1), (2, 3), (4, 5), (6, 7)]
Round 2
Start scores: {0: '0', 1: '1', 2: '0', 3: '1', 4: '0', 5: '1', 6: '0', 7: '1'}
Assigned pairs: [(0, 2), (1, 3), (4, 6), (5, 7)]
Round 3
Start scores: {0: '00', 1: '10', 2: '01', 3: '11', 4: '00', 5: '10', 6: '01', 7: '11'}
Assigned pairs: [(0, 4), (1, 5), (2, 6), (3, 7)]

Final scores: {0: '000', 1: '100', 2: '010', 3: '110', 4: '001', 5: '101', 6: '011', 7: '111'}
Tournament results: [7, 3, 5, 1, 6, 2, 4, 0]


[7, 3, 5, 1, 6, 2, 4, 0]

In [196]:
participants = (1, 7, 16, 0, 3, 5, 12, 223)
binary_tournament(list(participants), comparator, verbose=True)

Round 1
Start scores: {1: '', 7: '', 16: '', 0: '', 3: '', 5: '', 12: '', 223: ''}
Assigned pairs: [(1, 7), (16, 0), (3, 5), (12, 223)]
Round 2
Start scores: {1: '0', 7: '1', 16: '1', 0: '0', 3: '0', 5: '1', 12: '0', 223: '1'}
Assigned pairs: [(1, 0), (7, 16), (3, 12), (5, 223)]
Round 3
Start scores: {1: '01', 7: '10', 16: '11', 0: '00', 3: '00', 5: '10', 12: '01', 223: '11'}
Assigned pairs: [(1, 12), (7, 5), (16, 223), (0, 3)]

Final scores: {1: '010', 7: '101', 16: '110', 0: '000', 3: '001', 5: '100', 12: '011', 223: '111'}
Tournament results: [223, 16, 7, 5, 12, 1, 3, 0]


[223, 16, 7, 5, 12, 1, 3, 0]

## Binary tournament 

So here is how it works:

1. For each participant let's get a score label, before the first round it would be empty
2. When the round starts participants with the same labels get paired
3. After matches (comparisons) are finished match result is added to participant's label: 1 for win and 0 for lose

2 and 3 repeat until it was log2(N) rounds, by that time each score becomes unique and the most brilliant part: the score is the reverse of the place in binary
 
The thing is that instead of throwing off participants who were eliminated, we continue the tournament, but in separate groups: losers and the winners.
And the label identifies them explicitly. It also can be done implicitly through recursive splitting and merging the groups, but I find this example more representative. Anyway next code cell also implements an efficient version.

As I understand it's a variation on a [swiss tournament system](https://en.wikipedia.org/wiki/Swiss-system_tournament). I call this a *binary tournament*.

In [197]:
def binary_tournament_efficient(participants: Sequence[object], comparator: Callable[[object, object], bool]) -> Sequence[object]:
    count_check(participants)

    # actual bottleneck of the previous implementation is need to search by label, 
    # this implementation can just iterate on list and does not need labels

    def binary_tournament_aux(participants: Sequence[object]) -> Sequence[object]:
        winners = []
        losers = []

        for pair in range(len(participants) // 2):
            first = participants[pair * 2]
            second = participants[pair * 2 + 1]

            first_won = comparator(first, second)
            winners.append(first if first_won else second)
            losers.append(second if first_won else first)

        if len(participants) == 2:
            return winners + losers
        else:
            return binary_tournament_aux(winners) + binary_tournament_aux(losers)

    return binary_tournament_aux(participants)

In [198]:
comparator = lambda x, y: x > y

participants = range(8)
binary_tournament_efficient(list(participants), comparator)

[7, 3, 5, 1, 6, 2, 4, 0]

In [199]:

participants = (1, 7, 16, 0, 3, 5, 12, 223)
binary_tournament_efficient(list(participants), comparator)

[223, 16, 7, 5, 12, 1, 3, 0]

## Quantum binary tournament

But let's play with tournaments a bit more.

Another topic is tournament outcome calculation being complex enough to be considered a blackbox function - only way to know if somebody will win is to measure the outcome. Yes, we go quantum. 

What I'm interested in is augmenting the binary tournament with quantum circuits and any potential benefits of it.

As we are operating binary strings I find [Bernstein-Vazirani algorithm](https://learn.qiskit.org/course/ch-algorithms/bernstein-vazirani-algorithm) especially useful:
It allows us to find some hidden binary string s used by given black box function in 1 step.

Now let's speculate on which binary string need to be searched and how oracle gets to know it.

Our classic algorithm depends on black box comparator, so we can say that instead of comparator we can have black boxes for each matchup with encoded string being an outcome: 01 or 10. However, the comparator is also considered an O(1) operation, so there is no benefit.

On the other hand we can expect an ability to fix one of comparator arguments giving us n oracles which are capable of returning n-bit string of matchup results in O(1). So we can build pairwise comparisons matrix in n actions. But how fast can we move from matrix to binary tournament result?

In [200]:
def oracle_comparison(participants: Sequence[object], comparator: Callable[[object, object], bool]) -> Sequence[Sequence[bool]]:
    def get_hidden_string(participant: object) -> Sequence[bool]:
        return [*map(lambda p: str(int(comparator(participant, p))), participants)]
    
    # based on https://learn.qiskit.org/course/ch-algorithms/bernstein-vazirani-algorithm
    def build_oracle(s: str) -> QuantumCircuit:
        n = len(s)

        bv_circuit = QuantumCircuit(n+1, n)

        # put auxiliary in state |->
        bv_circuit.h(n)
        bv_circuit.z(n)

        # Apply Hadamard gates before querying the oracle
        for i in range(n):
            bv_circuit.h(i)
        
        # Apply barrier 
        bv_circuit.barrier()

        # Apply the inner-product oracle
        s = s[::-1] # reverse s to fit qiskit's qubit ordering
        for q in range(n):
            if s[q] == '0':
                bv_circuit.i(q)
            else:
                bv_circuit.cx(q, n)

        # Apply barrier 
        bv_circuit.barrier()

        #Apply Hadamard gates after querying the oracle
        for i in range(n):
            bv_circuit.h(i)

        # Measurement
        for i in range(n):
            bv_circuit.measure(i, i)

        return bv_circuit

    def get_comparisons(bv_circuit: QuantumCircuit, shots = 1024) -> Sequence[bool]:
        aer_sim = Aer.get_backend('aer_simulator')
        results = aer_sim.run(bv_circuit).result()
        counts = results.get_counts()

        row = max(counts, key=counts.get)
        return [e == '1' for e in row]


    hidden_strings = map(lambda p: get_hidden_string(p), participants)
    oracles = map(lambda s: build_oracle(s), hidden_strings)
    pairwise_comparisons = list(map(lambda o: get_comparisons(o), oracles))

    return pairwise_comparisons

In [201]:
participants = range(4)
comparator = lambda x, y: x > y

oracle_comparison(list(participants), comparator)

[[False, False, False, False],
 [True, False, False, False],
 [True, True, False, False],
 [True, True, True, False]]

Apparently, I can't think of a way faster than we needed to build a binary tournament from scratch: O(n log(n)). It can be done by reordering matrix columns and rows according to tournament state and reading the diagonal, supplying a lookup as a comparator to binary tournament, everything else - complexity is the same.  

In [202]:
def matrix_reordering(participants: Sequence[object], pairwise_comparisons: Sequence[Sequence[bool]]) -> Sequence[object]:
    participants_count, _ = count_check(participants)
    matrix = np.array(pairwise_comparisons)

    def reordering_aux(indices: Sequence[object]) -> Sequence[object]:
        slice = matrix.take(indices, axis=0).take(indices, axis=1)

        for pair in range(len(indices) // 2):
            first = pair * 2
            second = pair * 2 + 1

            slice[:, [first, second]] = slice[:, [second, first]]

        labels = slice.diagonal()
        indices_labeled = list(zip(indices, labels))

        winners = [p[0] for p in indices_labeled if p[1]]
        losers = [p[0] for p in indices_labeled if not p[1]]

        if len(indices) == 2:
            return winners + losers
        else:
            return reordering_aux(winners) + reordering_aux(losers)
    
    index = reordering_aux(range(participants_count))
    return np.array(participants)[index].tolist()

In [203]:
comparator = lambda x, y: x > y

participants = range(8)
pairwise_comparisons = oracle_comparison(list(participants), comparator)
matrix_reordering(participants, pairwise_comparisons)

[7, 3, 5, 1, 6, 2, 4, 0]

In [204]:
participants = (1, 7, 16, 0, 3, 5, 12, 223)
pairwise_comparisons = oracle_comparison(list(participants), comparator)
matrix_reordering(participants, pairwise_comparisons)

[223, 16, 7, 5, 12, 1, 3, 0]